In [1]:
import pandas as pd
import requests
from rich.progress import track
import ast
import numpy as np



In [2]:
df = pd.read_csv('swarfarm_spells.csv')

In [3]:
df

,name,description,slot,cooltime,hits,passive,aoe,random,multiplier_formula,multiplier_formula_raw,...,upgrade_4_effect,upgrade_4_amount,upgrade_5_effect,upgrade_5_amount,upgrade_6_effect,upgrade_6_amount,upgrade_7_effect,upgrade_7_amount,upgrade_8_effect,upgrade_8_amount
0,Forbidden Ingredient (Passive),Increases the Skill Level of the monster by 1 ...,1,0.0,0,True,False,False,NaN,[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,Special Ingredient (Passive),"The XP granted when used as Power-up material,...",1,0.0,0,True,False,False,NaN,[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,Best Ingredient (Passive),The XP granted will additionally increase by 5...,2,0.0,0,True,False,False,NaN,[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,Splinter Attack,Hurls sharp pieces of lumber and stuns the ene...,1,0.0,1,False,False,False,3.6*{ATK},"[[""ATK"", ""*"", 3.6]]",...,Damage +{0}%,10,Effect Rate +{0}%,10,Damage +{0}%,10,Effect Rate +{0}%,15,-1,-1
4,Splinter Attack,Hurls sharp pieces of lumber and stuns the ene...,1,0.0,1,False,False,False,3.6*{ATK},"[[""ATK"", ""*"", 3.6]]",...,Damage +{0}%,10,Effect Rate +{0}%,10,Damage +{0}%,10,Effect Rate +{0}%,15,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,Pulverize,Strikes the enemy with a hammer and prevents t...,1,0.0,1,False,False,False,2.4*{ATK} + 2.8*{DEF},"[[""ATK"", ""*"", 2.4], [""+""], [""DEF"", ""*"", 2.8]]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3796,Spinning Smash,"Swings the hammer, attacking all enemies to in...",2,3.0,1,False,True,False,3.7*{DEF},"[[""DEF"", ""*"", 3.7]]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3797,Powerful Smash,"Slams the enemy with a hammer, decreasing the ...",2,3.0,1,False,False,False,3.7*{ATK} + 5.0*{DEF},"[[""ATK"", ""*"", 3.7], [""+""], [""DEF"", ""*"", 5.0]]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3798,Spinning Smash,"Swings the hammer, attacking all enemies to in...",2,3.0,1,False,True,False,3.7*{DEF},"[[""DEF"", ""*"", 3.7]]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [4]:
liste = df['slot'].unique().tolist()
liste.append('*')
liste

[1, 2, 3, 4, -1, '*']

In [5]:
from types import NoneType


def extraire_variables_imbriquees(df, colonne, rename=None):
    
    df[colonne] = [ast.literal_eval(str(item)) for index, item in df[colonne].items()]
    
    df = pd.concat([df.drop([colonne], axis=1), df[colonne].apply(pd.Series)], axis=1)


    return df

def changement_effect(x, number):
    if x != 0 and x!= -1 and x != "-1":
        effect = {key + number: x[key] for key in x} # modification key
    else:
        effect = 0
    return effect

def changement_lead(x):
    if x != 0 and x!= -1 and x != "-1" and not isinstance(x, NoneType):
        effect = {key + '_leader': x[key] for key in x} # modification key
    else:
        effect = 0
    return effect

def swarfarm_monstres():
        # database swarfarm
    url = "https://swarfarm.com/api/v2/monsters/?page=1"
    r = requests.get(url=url)
    data = r.json()
    df = pd.DataFrame(data)

    for i in track(range(2,27), description="Chargement de la Database Swarfarm..."):
        url = f"https://swarfarm.com/api/v2/monsters/?page={i}"
        r = requests.get(url=url)
        data = r.json()
        df2 = pd.DataFrame(data)
        df = pd.concat([df, df2])
        
    
    # on extrait les variables du dict dans la colonne ['results'] et on supprime ce qui m'intéresse pas
    df = extraire_variables_imbriquees(df, 'results')
    df.drop(['next', 'previous'], axis=1, inplace=True)
    # On garde ce qui nous intéresse
    df_mobs_swarfarm = df[['id', 'url', 'com2us_id', 'family_id', 'name', 'awaken_level', 'element', 'archetype', 'skill_ups_to_max', 'max_lvl_hp', 'max_lvl_attack', 'max_lvl_defense', 'speed', 'crit_rate', 'crit_damage', 'resistance', 'accuracy']]
    
    # df_mobs_swarfarm['leader_skill'] = df_mobs_swarfarm['leader_skill'].apply(changement_lead)
    

    
    # on rajoute le type des monstres dans la colonne 'name' pour les monstres non-awaken
    
    df_mobs_swarfarm.reset_index(drop=True, inplace=True)
 

    
    # df_mobs_swarfarm.to_excel('swarfarm_stats.xlsx', index=False)
    # df_mobs_swarfarm.to_csv('swarfarm_stats.csv', index=False)
    
    return df_mobs_swarfarm
    

In [6]:
df_mob = swarfarm_monstres()

Output()

In [7]:
df_mob['skill_ups_to_max'].fillna(0, inplace=True)
df_mob['skill_ups_to_max'] = df_mob['skill_ups_to_max'].astype(int)

df_mob['element'] = df_mob['element'].astype('category')
df_mob['archetype'] = df_mob['archetype'].astype('category')

In [8]:
df_mob

,id,url,com2us_id,family_id,name,awaken_level,element,archetype,skill_ups_to_max,max_lvl_hp,max_lvl_attack,max_lvl_defense,speed,crit_rate,crit_damage,resistance,accuracy
0,1,https://swarfarm.com/api/v2/monsters/1/,12302,12300,Forest Keeper,0,Fire,Defense,7,6420.0,362.0,527.0,88,15,50,15,0
1,2,https://swarfarm.com/api/v2/monsters/2/,12702,12700,Ghost,0,Fire,Support,6,7740.0,516.0,285.0,101,15,50,15,0
2,3,https://swarfarm.com/api/v2/monsters/3/,13002,13000,Horned Frog,0,Fire,HP,5,8400.0,362.0,395.0,100,15,50,15,0
3,4,https://swarfarm.com/api/v2/monsters/4/,12902,12900,Mimick,0,Fire,Defense,6,5100.0,417.0,560.0,86,15,50,15,0
4,5,https://swarfarm.com/api/v2/monsters/5/,12402,12400,Mushroom,0,Fire,Attack,4,6915.0,505.0,351.0,104,15,50,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2548,2564,https://swarfarm.com/api/v2/monsters/2564/,121511,100000,2차각성 하르퓨 수정_오른쪽(물),0,Water,none,0,6585.0,439.0,439.0,110,15,50,15,0
2549,2565,https://swarfarm.com/api/v2/monsters/2565/,121512,100000,2차각성 하르퓨 수정_오른쪽(불),0,Fire,none,0,6585.0,439.0,439.0,110,15,50,15,0
2550,2566,https://swarfarm.com/api/v2/monsters/2566/,121513,100000,2차각성 하르퓨 수정_오른쪽(바람),0,Wind,none,0,6585.0,439.0,439.0,110,15,50,15,0
2551,2567,https://swarfarm.com/api/v2/monsters/2567/,121514,100000,2차각성 하르퓨 수정_오른쪽(빛),0,Light,none,0,6585.0,439.0,439.0,110,15,50,15,0


In [9]:
# df_mob[df_mob['id'].isin(list_mob)]

In [10]:
from sqlalchemy import create_engine, text, Float, BigInteger
from os import environ

# https://betterprogramming.pub/how-to-execute-plain-sql-queries-with-sqlalchemy-627a3741fdb1

DB = environ.get('API_SQL')

def init_connection():
    engine = create_engine(DB,
                           echo=False,
                           connect_args={'options': '-csearch_path={}'.format('sw')})
    return engine.connect()

conn = init_connection()

In [11]:
df_mob.drop(['url', 'name', 'element', 'archetype'], axis=1).set_index('id').to_sql('sw_ref_monsters_stats', conn, if_exists='replace')

553

In [12]:
conn.commit()